# Data source: Global Energy Monitor

In [14]:
import pandas as pd

df_Units = pd.read_csv('../Units.csv')
df_Buses = pd.read_csv('../Buses.csv')
df_Units_modified = df_Units[(df_Units['type'] != 'interconnector') & (df_Units['type'] != 'demand_flex')]
df_Units_modified

,UnitID,name,type,Bus name,x,y,capacity,Technology,Cost
5,6,BFSE01,battery,way/90534270-400,-1.22,52.585,50.0,battery storage,0.00
6,7,AG-HLIM01,battery,way/263863442-400,-1.494,51.598,0.0,battery storage,0.00
7,8,AG-HLIM02,battery,way/4715504-400,-1.194,50.852,60.0,battery storage,0.00
8,9,AG-HLIM03,battery,way/95279334-400,-1.99,51.616,60.0,battery storage,0.00
9,10,AG-HLIM04,battery,way/263863442-400,-1.494,51.598,100.0,battery storage,0.00
...,...,...,...,...,...,...,...,...,...
1051,1052,GRGBW-2,offwind,way/576238223-400,1.713867,52.066,1075.0,offwind,0.02
1052,1053,GRGBW-3,offwind,way/576238223-400,1.713867,52.066,522.0,offwind,0.02
1053,1054,SGRWO-1,offwind,GB7-275,-1.741,56.588,36.9,offwind,0.02
1054,1055,ABRBO-1,offwind,GB7-275,-1.98333,57.2167,96.8,offwind,0.02


## Bioenergy

### Select datas from UK

In [24]:
df_bioenergy = pd.read_excel('Global-Bioenergy-Power-Tracker-GBPT-September-2024.xlsx', sheet_name = 'Data')
df_bioenergy = df_bioenergy[df_bioenergy['Country/Area'] == 'United Kingdom']
df_bioenergy = df_bioenergy[['Longitude', 'Latitude', 'Capacity (MW)']]

df_Units_bioenergy = df_Units_modified[df_Units_modified['type'] == 'biomass']
df_Units_bioenergy

,UnitID,name,type,Bus name,x,y,capacity,Technology,Cost
23,24,DRAXD-2,biomass,way/160580052-400,-0.996389,53.7358,39.0,biomass,14.893617
85,86,MARK-1,biomass,way/1185489725-220,-3.603516,57.480403,0.0,biomass,14.893617
102,103,TIBG-1,biomass,way/94530230-275,0.389,51.459,10.0,biomass,14.893617
783,784,DRAXX-1,biomass,GB30-400,-0.626221,53.748711,440.0,biomass,14.893617
784,785,DRAXX-2,biomass,GB30-400,-0.626221,53.748711,200.5,biomass,14.893617
785,786,DRAXX-3,biomass,GB30-400,-0.626221,53.748711,200.5,biomass,14.893617
786,787,DRAXX-4,biomass,GB30-400,-0.626221,53.748711,126.0,biomass,14.893617
787,788,DRAXX-5,biomass,GB30-400,-0.626221,53.748711,42.0,biomass,14.893617
788,789,DRAXX-6,biomass,GB30-400,-0.626221,53.748711,0.0,biomass,14.893617


### Map units based on lon and lat

In [5]:
len(df_bioenergy)

60

In [7]:
len(df_Units_modified[df_Units_modified['type'] == 'biomass'])

9

In [ ]:
# import folium

# map_GB = folium.Map(location=[54, -2], zoom_start=6, tiles="CartoDB positron")
# for _, row in df_bioenergy.iterrows():

#     popup_html = f"""
#     <b>Power station:</b> {row.get("Project Name", "")}<br>
#     <b>x:</b> {row.get("Longitude", "")}<br>
#     <b>y:</b> {row.get("Latitude", "")}
#     """

#     folium.CircleMarker(
#             location=[row["Latitude"], row["Longitude"]],
#             radius=2,
#             color="yellow",
#             fill=True,
#             fill_opacity=0.7,
#             popup=folium.Popup(popup_html, max_width=200)
#         ).add_to(map_GB)


# for _, row in df_Units_bioenergy.iterrows():

#     popup_html = f"""
#     <b>UnitID:</b> {row.get("UnitID", "")}<br>
#     <b>x:</b> {row.get("x", "")}<br>
#     <b>y:</b> {row.get("y", "")}
#     """

#     folium.CircleMarker(
#             location=[row["y"], row["x"]],
#             radius=2,
#             color="purple",
#             fill=True,
#             fill_opacity=0.7,
#             popup=folium.Popup(popup_html, max_width=200)
#         ).add_to(map_GB)

# for _, row in df_Buses.iterrows():

#     popup_html = f"""
#     <b>UnitID:</b> {row.get("BusID", "")}<br>
#     <b>x:</b> {row.get("x", "")}<br>
#     <b>y:</b> {row.get("y", "")}
#     """

#     folium.CircleMarker(
#             location=[row["y"], row["x"]],
#             radius=2,
#             color="red",
#             fill=True,
#             fill_opacity=0.7,
#             popup=folium.Popup(popup_html, max_width=200)
#         ).add_to(map_GB)

# map_GB.save('Biomass_Visualization.html')

In [29]:
import numpy as np
from scipy.spatial import cKDTree

# Build KDTree from bioenergy points
tree = cKDTree(df_bioenergy[["Longitude", "Latitude"]].to_numpy())

# Query nearest bioenergy point for each unit
dist, idx = tree.query(df_Units_bioenergy[['x', 'y']].to_numpy(), k=1)

# Assign the nearest df_bioenergy index to each unit
df_Units_bioenergy = df_Units_bioenergy.copy()
df_Units_bioenergy["nearest_bioenergy_idx"] = idx
df_Units_bioenergy["nearest_bioenergy_lon"] = df_bioenergy.iloc[idx]["Longitude"].values
df_Units_bioenergy["nearest_bioenergy_lat"] = df_bioenergy.iloc[idx]["Latitude"].values

df_Units_bioenergy

,UnitID,name,type,Bus name,x,y,capacity,Technology,Cost,nearest_bioenergy_idx,nearest_bioenergy_lon,nearest_bioenergy_lat,distance_to_bioenergy
23,24,DRAXD-2,biomass,way/160580052-400,-0.996389,53.7358,39.0,biomass,14.893617,7,-0.999021,53.737196,0.002979
85,86,MARK-1,biomass,way/1185489725-220,-3.603516,57.480403,0.0,biomass,14.893617,51,-2.365300,57.257100,1.258190
102,103,TIBG-1,biomass,way/94530230-275,0.389,51.459,10.0,biomass,14.893617,52,0.351500,51.453500,0.037901
783,784,DRAXX-1,biomass,GB30-400,-0.626221,53.748711,440.0,biomass,14.893617,28,-0.701200,53.619800,0.149130
784,785,DRAXX-2,biomass,GB30-400,-0.626221,53.748711,200.5,biomass,14.893617,28,-0.701200,53.619800,0.149130
785,786,DRAXX-3,biomass,GB30-400,-0.626221,53.748711,200.5,biomass,14.893617,28,-0.701200,53.619800,0.149130
786,787,DRAXX-4,biomass,GB30-400,-0.626221,53.748711,126.0,biomass,14.893617,28,-0.701200,53.619800,0.149130
787,788,DRAXX-5,biomass,GB30-400,-0.626221,53.748711,42.0,biomass,14.893617,28,-0.701200,53.619800,0.149130
788,789,DRAXX-6,biomass,GB30-400,-0.626221,53.748711,0.0,biomass,14.893617,28,-0.701200,53.619800,0.149130
